## **Loading the dataset Tiny_imagenet**

1. load using numpy and torch
2. the input is 64x64 pixel coloured image (i.e. 64x64x3 size input)

In [1]:
print(3)

3


## **Training 2 different models**
1. We aim to make 2 similar models with different architechture, with different activation function, with the aim to explore the vanishing gradients problem.
2. we will also look at the weight updates in between training to quantify the above problem.

In [ ]:
import numpy as np
import pytorch as torch

## **Ablation Study**